# Psuedo gene detection test 

Software: https://github.com/filip-husnik/pseudofinder 

Install via https://github.com/filip-husnik/pseudofinder/wiki/2.-Installing-Pseudofinder#easy-installation 

I had to modify the setup.sh to the following:

```bash
#!/usr/bin/env bash

# setting colors to use
GREEN='\033[0;32m'
RED='\033[0;31m'
NC='\033[0m'
PATH_TO_PSEUDOFINDER=`dirname $0`

printf "\n    ${GREEN}Setting up conda environment...${NC}\n\n"

chmod +x $PATH_TO_PSEUDOFINDER/pseudofinder.py

## creating environment and installing dependencies
mamba env create --file $PATH_TO_PSEUDOFINDER/modules/environment.yml 

## activating environment
source activate pseudofinder

## creating directory for conda-env-specific source files
mkdir -p ${CONDA_PREFIX}/etc/conda/activate.d

## adding codeml-2.ctl file path:
echo '#!/bin/sh'" \

export PATH=\"$(pwd):"'$PATH'\"" \

export ctl=\"$(pwd)/codeml-2.ctl\"" >> ${CONDA_PREFIX}/etc/conda/activate.d/env_vars.sh

# re-activating environment so variable and PATH changes take effect
source activate pseudofinder

printf "\n        ${GREEN}DONE!${NC}\n\n"

# to reset:
# conda env remove --name pseudofinder
```

*This really should just be in Bioconda* 


## Getting started

This program needs you to give it a database of true genes (as amino acid) to compare with. I am using the wgMLST gene panel from Enterobase.

```
wget https://enterobase.warwick.ac.uk/schemes/Salmonella.wgMLST/exemplar.alleles.fasta.gz 
```

These are in nucleotide, so we must convert nuc to aa. 

In [1]:
from Bio import SeqIO, Seq
import gzip 
from Bio.Data.CodonTable import TranslationError 

gfile = gzip.open("exemplar.alleles.fasta.gz", "rt")

out_file = open('sal_alleles.faa', 'w') 


number_skip = 0 
for record in SeqIO.parse(gfile, 'fasta'):
    try:
        record.seq = record.seq.translate(cds=True)
        out_file.write(record.format("fasta"))        
    except TranslationError:
        number_skip += 1
print(f'Could not translate {number_skip} seqs ')
        


Could not translate 1596 seqs 


In [2]:
!head sal_alleles.faa
!tail sal_alleles.faa
!cat  sal_alleles.faa | grep '>' | wc -l 

>STMMW_00651_1
MHEAQIRVAIAGAGGRMGRQLIQAAMAMEGVQLGAALEREGSSLLGSDAGELAGAGKSGV
IVQSSLEAVKDDFDVFIDFTRPEGTLTHLAFCRQHGKGMVIGTTGFDDAGKQAIREASQE
IAIVFAANFSVGVNVMLKLLEKAAKVMGDYSDIEIIEAHHRHKVDAPSGTALAMGEAIAG
ALDKNLKDCAVYSREGYTGERVAGTIGFATVRAGDIVGEHTAMFADIGERVEITHKASSR
MTFANGALRSALWLKTKKNGLFDMRDVLGLDVL
>STMMW_00121_1
MAKRDYYEILGVSKTAEEREIKKAYKRLAMKYHPDRNQGDKEAEAKFKEIKEAYEVLTDA
QKRAAYDQYGHAAFEQGGMGGGFGGGFNGGADFSDIFGDVFGDIFGGGRGRQRAARGADL
RYNMDLTLEEAVRGVTKEIRIPTLEECDVCHGSGAKAGTQPQTCPTCHGSGQVQMRQGFF
QDALAILRNKLVVREHYLPCVLFGDDAPTEFTVGPVTFTQNAMFFRDKKSVFRHSVDINT
NAHIKSVTSAITQGFFRENVPTPDESRKFVGEFQKRAIKIYKDYPWVASIKVTDCDEVTS
QERAIQATELAIHIIRILLGAEPTRKIRLAWSRSNALNTAHLYSDADGVIHASVGANSLG
PVGIINWYKALMKCDLELEILGSALLPIVNPIETNHLHQRLIDAINWFGDAATDSNPSSS
IVKYVSAIERLFFGKFESGRTKVFAGRIKYILDAFGCDGDHQVYDQALKVYRARSILVHG
EIYQTEDEANESICLASSLSRMCLLCSAQLYSMMQNAFDNPDALALEEIMKRIGAEGLDW
LVDAAGFHK
>ZV79_RS12785_1
MKVETISYVKKNAATLDLSEPILVTQNGVPAYVIESYDQQQERENAIALLKLLTLSEKDK
AEGRVFSKDQLLDSLED
19466


This needs to be made into a BLAST database.

In [3]:
!makeblastdb -in sal_alleles.faa -dbtype prot



Building a new DB, current time: 07/12/2022 23:01:06
New DB name:   /home/ubuntu/code/journal/pseudo/sal_alleles.faa
New DB title:  sal_alleles.faa
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 19466 sequences in 0.440682 seconds.




We also need a genome to query. I selected a random genome from EnteroBase (SAL_VC3907AA_AS) using the pregenerated annotation (gbk). 

# Running pseudofinder



In [40]:
!pseudofinder.py annotate -g SAL_VC3907AA_AS.enterobase.gbk -db sal_alleles.faa -op test 

2022-07-12 22:41:45	CDS extracted from:			SAL_VC3907AA_AS.enterobase.gbk
			Written to file:			test_cds.fasta.
2022-07-12 22:41:45	Intergenic regions extracted from:	SAL_VC3907AA_AS.enterobase.gbk
			Written to file:			test_intergenic.fasta.
2022-07-12 22:41:45	blastp executed with 4 threads on test_proteome.faa.
2022-07-12 22:43:07	blastx executed with 4 threads on test_intergenic.fasta.
2022-07-12 22:43:30	Checking contig 1 / 62 for pseudogenes.
2022-07-12 22:43:30	Number of ORFs on this contig: 624
			Number of pseudogenes flagged: 41
2022-07-12 22:43:30	Checking contig 2 / 62 for pseudogenes.
2022-07-12 22:43:30	Number of ORFs on this contig: 582
			Number of pseudogenes flagged: 43
2022-07-12 22:43:30	Checking contig 3 / 62 for pseudogenes.
2022-07-12 22:43:30	Number of ORFs on this contig: 459
			Number of pseudogenes flagged: 42
2022-07-12 22:43:30	Checking contig 4 / 62 for pseudogenes.
2022-07-12 22:43:30	Number of ORFs on this contig: 462
			Number of pseudogenes flagged: 36


In [41]:
!ls

SAL_VC3907AA_AS.enterobase.gbk	test_contigs.fasta
exemplar.alleles.fasta.gz	test_intact.faa
pseudofinder			test_intact.ffn
psuedo-work.ipynb		test_intact.gff
sal_alleles.faa			test_interactive_results.html
sal_alleles.faa.dmnd		test_intergenic.fasta
sal_alleles.faa.pdb		test_intergenic.fasta.blastX_output.tsv
sal_alleles.faa.phr		test_log.txt
sal_alleles.faa.pin		test_map.pdf
sal_alleles.faa.pot		test_proteome.faa
sal_alleles.faa.psq		test_proteome.faa.blastP_output.tsv
sal_alleles.faa.ptf		test_pseudos.fasta
sal_alleles.faa.pto		test_pseudos.gff
test_cds.fasta
